In [1]:
#Import API
from dora.api import DataExplorer

import pandas as pd
from datetime import date, timedelta, datetime
from matplotlib import pyplot
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from uszipcode import ZipcodeSearchEngine
import numpy as np

import random
import time
from sklearn.model_selection import KFold
import json
import sys

explorer = DataExplorer()

In [3]:
def custPerCluster(df):
    fig=plt.figure()
    maxn=df['cluster'].max()
    plt.hist(df['cluster'].values)
    plt.xlabel('Clusters')
    plt.ylabel('# of Customers')
    plt.xticks(range(0,maxn+1))
    plt.title('Number of Customers per Cluster')
    return fig

In [4]:
def clusterDist(df):
    maxn=df['cluster'].max()
    clusterStats=pd.DataFrame()
    for i in range(maxn+1):
        mask=(df['cluster']==i)
        t=df.loc[mask]
        clusterStats.loc[i,'avgNumOrders']=t['numorders'].mean()
        clusterStats.loc[i,'avgTotalSpent']=t['totalspent'].mean()
        clusterStats.loc[i,'numHouseholds']=t['householdid'].nunique()
        clusterStats.loc[i,'numZipcoes']=t['zipcode'].nunique()
        clusterStats.loc[i,'avgTotalPopMales']=t['totalmales'].mean()
        clusterStats.loc[i,'avgTotalPopFemales']=t['totalfemales'].mean()
        clusterStats.loc[i,'avgTotalPop']=t['totalpop'].mean()
        clusterStats.loc[i,'medianAge']=t['medianage'].median()
    return clusterStats

In [5]:
def clusterZips(df):
    maxn=df['cluster'].max()
    search = ZipcodeSearchEngine()
    zipstates={}
    zipcities={}
    for i in range(maxn+1):
        #print ("Cluster "+str(i))
        states=set()
        cities=set()
        mask=(df['cluster']==i)
        t=df[['zipcode','cluster']].loc[mask]
        zipcodes=t['zipcode'].unique()
        for j in range(len(zipcodes)):
            city=search.by_zipcode(str(zipcodes[j]))
            cities.add(city['City'])
            states.add(city['State'])
        zipstates.update({i:states})
        zipcities.update({i:cities})
    return zipstates, zipcities

statsByCustomer Query

In [6]:
statsByCust=explorer.customers.statsByCustomer()
statsByCustomer=pd.DataFrame(statsByCust.results, columns=statsByCust.columns)
statsByCustomer.head()

,customermatchedid,numorders,gender,zipcode,totalpop,medianage,totalmales,totalfemales,totalspent,householdid,firstname,numcustomerid
0,37917,412,0,10036,22413,38.8,12687,9726,606.25,19885296,MIKE,412
1,37907,189,1,10036,22413,38.8,12687,9726,651.59,19885296,HILDA,189
2,140299,99,0,10036,22413,38.8,12687,9726,0.00,49927024,MIKE,99
3,37901,99,2,10036,22413,38.8,12687,9726,2226.32,19885296,,99
4,140298,70,0,10036,22413,38.8,12687,9726,0.00,49927024,JIM,70


Use the API to get 10 clusters of customers. Bad zipcodes have been removed from the dataset. Customers with no gender assigned are within the dataset of customers to be clustered. A specified feature_set is not defined by the user, neither are the features that will be used for the clustering. In this case, the feature_set will be the data from statsByCustomer and the features will be [numOrders, gender, totalpop, totalspent]. 

In [7]:
#Get 10 clusters and print out the head of the dataframe
cCluster=explorer.customers.clusterCustomers(n_clusters=10)

In [8]:
#Lists the columns of the cCluster
cCluster.columns

array(['numorders', 'gender', 'totalpop', 'totalspent', 'zipcode',
       'medianage', 'totalmales', 'totalfemales', 'customermatchedid',
       'householdid', 'firstname', 'numcustomerid', 'cluster',
       'customerids'], dtype=object)

Cluster the customers but remove certain householdids. Specifiy the features that will be used in the clustering. 

In [13]:
#Elimate householdids from the data that will be clustered
response=explorer.customers.statsByCustomer(householdid=['19885296','49927024'])

In [15]:
#Cluster the results by customer
cCluster_rmHID=explorer.customers.clusterCustomers(feature_set=response, n_clusters=7, 
                                                   cluster_on=['medianage','totalmales','totalfemales','totalpop'])

In [16]:
#Get cluster stats
df=pd.DataFrame(cCluster_rmHID.results, columns=cCluster_rmHID.columns )
stats=clusterDist(df)
stats

,avgNumOrders,avgTotalSpent,numHouseholds,numZipcoes,avgTotalPopMales,avgTotalPopFemales,avgTotalPop,medianAge
0,1.188981,82.633711,36335.0,6977.0,9105.157723,9626.420413,18731.578137,41.3
1,1.200593,84.174745,10103.0,3821.0,30494.867450,33519.575997,64014.443447,38.3
2,1.213033,82.741326,3021.0,1798.0,42834.013848,48141.761292,90975.775140,37.7
3,1.192670,84.374738,24242.0,5864.0,19057.873589,20451.975210,39509.848800,39.0
4,1.180241,85.926228,30588.0,6572.0,3800.963022,3933.220112,7734.183134,43.0
5,1.191487,81.711357,35394.0,6929.0,13864.119790,14798.973879,28663.093669,40.0
6,1.192734,84.545345,14236.0,4647.0,24449.884134,26162.323893,50612.208027,36.3


In [17]:
cCluster_rmHID.columns

array(['medianage', 'totalmales', 'totalfemales', 'totalpop',
       'customermatchedid', 'numorders', 'gender', 'zipcode', 'totalspent',
       'householdid', 'firstname', 'numcustomerid', 'cluster',
       'customerids'], dtype=object)

In [98]:
df[(df['cluster']==1)]

,medianage,totalmales,totalfemales,totalpop,customermatchedid,numorders,gender,zipcode,totalspent,householdid,firstname,numcustomerid,cluster,customerids
4,42.0,31484.0,38261.0,69745.0,32345.0,17.0,0.0,44122,3757.26,19626230.0,JEROME,17.0,1,"(78833, 78839, 78837, 78838, 78836, 78844, 788..."
21,38.9,25949.0,33494.0,59443.0,140553.0,12.0,0.0,10301,319.95,50072776.0,ROBERT,12.0,1,"(13505, 13506, 13503, 13498, 13497, 13499, 134..."
32,39.4,30654.0,32510.0,63164.0,126164.0,11.0,0.0,18017,5253.00,36419822.0,STEPHEN,11.0,1,"(137696, 137697, 137695, 137694, 137693, 13770..."
37,38.1,34128.0,38306.0,72434.0,108633.0,10.0,0.0,17870,286.64,36201352.0,ROBERT,10.0,1,"(108363, 108366, 108367, 108365, 108368, 10836..."
58,35.9,27314.0,34539.0,61853.0,16104.0,9.0,0.0,21210,1927.50,18861947.0,GEORGE,9.0,1,"(63024, 63025, 63023, 63022, 63027, 63026, 630..."
107,39.6,31390.0,37981.0,69371.0,143961.0,8.0,1.0,10036,372.12,52160470.0,EDITH,8.0,1,"(172433, 172440, 172439, 172436, 172435, 17243..."
126,38.9,25949.0,33494.0,59443.0,16708.0,7.0,0.0,33463,336.53,18892135.0,ROBERT,7.0,1,"(16353, 16355, 16354, 16359, 16358, 16357, 16356)"
130,41.5,27848.0,31143.0,58991.0,18008.0,7.0,0.0,20016,643.92,18949555.0,JULES,7.0,1,"(47079, 47081, 47084, 47082, 47083, 47078, 47080)"
131,42.0,31484.0,38261.0,69745.0,45153.0,7.0,1.0,10024,2580.00,20235435.0,BARBARA,7.0,1,"(40626, 40629, 40627, 40625, 40623, 40624, 406..."
148,41.2,28799.0,31648.0,60447.0,22882.0,7.0,2.0,48215,385.98,19190028.0,N.,7.0,1,"(47320, 47322, 47318, 47319, 47317, 47321, 47323)"


In [19]:
# Create co-occurrence matrix

def create_cocmatrix(subset_matrix_cust_order):
    rows, cols = subset_matrix_cust_order.shape
    m = np.zeros((cols,cols))
    
    for i in range(cols):
        t = np.sum(subset_matrix_cust_order[subset_matrix_cust_order[:,i] > 0],axis=0)
        t[i] = 0
        m[i,:] = t
                    
    return m

In [20]:
def create_seed(rand):
    np.random.seed(rand) # create seed for repeatable results

In [21]:
# generate recommendations
# purchase_vec = customer purchases (vector)
# cocm = co occurrence matrix [items x items]
# num_rec = number of recommendations
def gen_recom(purchase_list,cocm,num_rec):
    rowsum = np.zeros(cocm.shape[0])
    
    for p in purchase_list:
        rowsum += cocm[p,:]
        
    rowsum[purchase_list] = 0
    indices = np.nonzero(rowsum)[0]
    toprec = indices[np.argsort(rowsum[indices])][-1 * num_rec:][::-1]
    return toprec

In [22]:
def get_indices(dataset, col, var=None):
    if var is None:
        return [y for y, x in enumerate(dataset[:,col])] # all
    
    return [y for y, x in enumerate(dataset[:,col]) if x == var]

In [23]:
def check_idx_size(idxlist,maximum_rows):
    if len(idxlist) > maximum_rows:
        return np.random.choice(idxlist, maximum_rows, replace=False)
    
    return idxlist

In [24]:
# customer to order lookup
matrix_co = np.load('cust_item_matrix.npy')
print ("created customer to order lookup")

matrix_co.shape

created customer to order lookup


(189559, 3990)

In [25]:
custlist = np.load('custids.npy')
print ("created customer id lookup")

custlist.shape

created customer id lookup


(189559,)

In [26]:
unique_clusters = sorted(df['cluster'].unique())
print (unique_clusters)

[0, 1, 2, 3, 4, 5, 6]


In [27]:
# bring in customerids to map with cluster ids
arr_cxd = np.zeros((custlist.shape[0],1))

In [28]:
arr_cxd.shape

(189559, 1)

In [110]:
# mapping cluster ids
for i in range(df.shape[0]):
    try:
        if len(df.loc[i,'customerids']) > 0:       
            for c in df.loc[i,'customerids']:
                idx = np.where(custlist==c)
                cid = df.loc[i,'cluster']
                arr_cxd[idx] = cid+1 # increment by 1 so we can reserve 0 for customers with no cluster id
    except:
        continue

In [57]:
maxn=df['cluster'].max()
print (maxn)

6


In [111]:
clusternames = dict()
for i in range(arr_cxd.shape[0]):    
    if arr_cxd[i][0] in clusternames:
        clusternames[arr_cxd[i][0]]+=1
        continue
        
    clusternames.update({arr_cxd[i][0]:0})
    
clusternames

{0.0: 5720,
 1.0: 43367,
 2.0: 12176,
 3.0: 3674,
 4.0: 29019,
 5.0: 36254,
 6.0: 42349,
 7.0: 16992}

In [58]:
unique_clusters = unique_clusters + [maxn+1]
print (unique_clusters)

[0, 1, 2, 3, 4, 5, 6, 7]


In [112]:
#80/20, 80 for 10-fold cross validation, 20 for test set holdout
create_seed(0)
indices = np.random.permutation(matrix_co.shape[0])
tsize = int(matrix_co.shape[0]*.8)

In [113]:
# customer to order matrix
matrix_co_training_idx, matrix_co_testing_idx  = indices[:tsize], indices[tsize:]
matrix_co_training, matrix_co_testing = matrix_co[matrix_co_training_idx,:], matrix_co[matrix_co_testing_idx,:]

In [114]:
# demo matrix
arr_cxd_training_idx, arr_cxd_testing_idx = indices[:tsize], indices[tsize:]
arr_cxd_training, arr_cxd_testing = arr_cxd[arr_cxd_training_idx,:], arr_cxd[arr_cxd_testing_idx,:]

In [115]:
matrix_co_training.shape, arr_cxd_training.shape, matrix_co.shape, arr_cxd.shape

((151647, 3990), (151647, 1), (189559, 3990), (189559, 1))

In [116]:
# put matrixes in to get cross validation recall
# generate recommendations
# mco = matrix of customer to order [customer x items]
# num_rec = number of recommendations
# num_folds = k number of folds for cross validation
# recall_remove = removed number from purchase history
def collab_recall_validation_tester(mco,num_rec,num_folds,recall_remove):
    # cross validation n-folds
    start_time = time.time()
    kf = KFold(n_splits=num_folds)
    list_total_acc=[]
    k_index = 0
    
    for train, test in kf.split(mco):
        print("start new k={0},train set={1},validation set={2}".format(k_index,mco[train].shape[0],mco[test].shape[0]))
        
        list_sub_acc = []
        # build co-occurrence matrix
        coo_matrix = create_cocmatrix(mco[train])

        # loop through test set
        # for each customer in test pool
        for i in range(mco[test].shape[0]):
            
            p_vec = mco[test][i,:]
            list_original_purchases = np.where(p_vec > 0)[0]
            # only run tests on customers with > recall_remove purchases for prediction
            if len(list_original_purchases) > recall_remove:
                # randomly select indexes to leave out
                list_removed_purchases = random.sample(list(list_original_purchases),recall_remove)
                # remove
                list_modified_purchases = list(set(list_original_purchases) - set(list_removed_purchases))
                # get sum all purchases except ones left out
                list_summed_coo_vec = gen_recom(list_modified_purchases,coo_matrix,num_rec)
                # check if return recommendations are in list
                list_recommended_match = set(list_removed_purchases) & set(list_summed_coo_vec)
                acc = len(list_recommended_match)/float(len(list_removed_purchases))
                list_sub_acc.append(acc)
                
        mean_sub_acc = np.mean(list_sub_acc)
        print ("** number of elements in calculation {0}".format(len(list_sub_acc)))
        #print list_sub_acc
        print ("** average fold accuracy {0}".format(mean_sub_acc))
        list_total_acc.append(mean_sub_acc)
        k_index += 1
        print ("**time elapsed {0}".format((time.time() - start_time)))
        print ("------------------------------------end")
        
    print ("list of total accuracy for each fold {0}".format(list_total_acc))
    average = np.mean(list_total_acc)
    print ("{0}-fold total average {1}".format(num_folds,average))
    print ("time elapsed {0}".format((time.time() - start_time)))
    return average

In [117]:
# put matrixes in to get cross validation coverage
# generate recommendations
# mco = matrix of customer to order [customer x items]
# num_rec = number of recommendations
# num_folds = k number of folds for cross validation
# recall_remove = removed number from purchase history
# list_catid = category customer is interested in
def collab_catalog_validation_tester(mco,num_rec,num_folds,recall_remove):
    # cross validation n-folds
    start_time = time.time()
    kf = KFold(n_splits=num_folds)
    list_total_coverage=[]
    k_index = 0
    
    for train, test in kf.split(mco):
        set_products = set()
        # build co-occurrence matrix
        print("start new k={0},train set={1},validation set={2}".format(k_index,mco[train].shape[0],mco[test].shape[0]))
        coo_matrix = create_cocmatrix(mco[train])

        # loop through test set
        # for each customer in test pool
        for i in range(mco[test].shape[0]):
            # purchase vector
            p_vec = mco[test][i,:]
            list_original_purchases = np.where(p_vec > 0)[0]
            # get sum all purchases
            list_summed_coo_vec = gen_recom(list_original_purchases,coo_matrix,num_rec)    
            set_products = set_products | set(list_summed_coo_vec)
                        
        mean_sub_cov = float(len(set_products))/p_vec.shape[0]
        print ("**length fold set {0}".format(float(len(set_products))))
        print ("**average fold coverage {0}".format(mean_sub_cov))
        list_total_coverage.append(mean_sub_cov)
        k_index += 1
        print ("**time elapsed {0}".format((time.time() - start_time)))
        print ("------------------------------------end")
    
    print ("list of total coverage for each fold {0}".format(list_total_coverage))
    average = np.mean(list_total_coverage)
    print ("{0}-fold total average {1}".format(num_folds,average))
    print ("time elapsed {0}".format((time.time() - start_time)))
    return average

In [118]:
def demo_filter(mco,mdo,clusters,maximum_rows,minimum_rows,num_rec,num_folds,recall_remove):
    accuracy = []
    coverage = []
    
    for c in clusters:
        #if r == 0: # 0 means not found in the cluster
        #    ridx = get_indices(mdo, 0) # use all regions
        #else:   
        cidx = get_indices(mdo, 0, c) # look for cluster in col 0

        if len(cidx) == 0:
            print ("cluster: {0} not found".format(c))
            continue

        if len(cidx) < minimum_rows:
            print ("too little data with cluster {0} found".format(c))
            continue

        cidx = check_idx_size(cidx,maximum_rows) # limit size
        print ("\ncluster: {0}, size: {1}, {2}".format(c, len(cidx), len(mco)))
                             
        total_accuracy = collab_recall_validation_tester(mco[cidx],num_rec,num_folds,recall_remove)
        accuracy.append((c,mco[cidx].shape[0],total_accuracy))
        total_coverage = collab_catalog_validation_tester(mco[cidx],num_rec,num_folds,recall_remove)
        coverage.append((c,mco[cidx].shape[0],total_coverage))
        
    return accuracy, coverage

In [119]:
# General test
seed = 0
create_seed(seed)
maximum_rows = 25000
minimum_rows = 1000

# test set
#randidx = np.random.choice(matrix_co_training.shape[0], 10000, replace=False)
#demo_results = demo_filter(matrix_co_training[randidx],arr_cxd_training[randidx],\
#                           regions,maximum_rows,minimum_rows,10,10,1)

# big set
demo_accuracy, demo_coverage = demo_filter(matrix_co_training,arr_cxd_training,\
                                   unique_clusters,maximum_rows,minimum_rows,10,10,1)


cluster: 0, size: 4578, 151647
start new k=0,train set=4120,validation set=458
** number of elements in calculation 135
** average fold accuracy 0.5703703703703704
**time elapsed 3.185616970062256
------------------------------------end
start new k=1,train set=4120,validation set=458
** number of elements in calculation 129
** average fold accuracy 0.5426356589147286
**time elapsed 6.07878303527832
------------------------------------end
start new k=2,train set=4120,validation set=458
** number of elements in calculation 132
** average fold accuracy 0.5
**time elapsed 9.27619481086731
------------------------------------end
start new k=3,train set=4120,validation set=458
** number of elements in calculation 142
** average fold accuracy 0.4788732394366197
**time elapsed 12.438875913619995
------------------------------------end
start new k=4,train set=4120,validation set=458
** number of elements in calculation 127
** average fold accuracy 0.4409448818897638
**time elapsed 15.560137987

**length fold set 672.0
**average fold coverage 0.16842105263157894
**time elapsed 605.663948059082
------------------------------------end
start new k=7,train set=22500,validation set=2500
**length fold set 677.0
**average fold coverage 0.16967418546365914
**time elapsed 703.5657501220703
------------------------------------end
start new k=8,train set=22500,validation set=2500
**length fold set 689.0
**average fold coverage 0.17268170426065163
**time elapsed 799.1413559913635
------------------------------------end
start new k=9,train set=22500,validation set=2500
**length fold set 717.0
**average fold coverage 0.17969924812030075
**time elapsed 892.6288068294525
------------------------------------end
list of total coverage for each fold [0.16541353383458646, 0.16516290726817043, 0.1649122807017544, 0.16516290726817043, 0.16340852130325814, 0.17092731829573934, 0.16842105263157894, 0.16967418546365914, 0.17268170426065163, 0.17969924812030075]
10-fold total average 0.1685463659147869

**length fold set 276.0
**average fold coverage 0.06917293233082707
**time elapsed 3.7492380142211914
------------------------------------end
start new k=2,train set=2630,validation set=292
**length fold set 257.0
**average fold coverage 0.06441102756892231
**time elapsed 5.592168092727661
------------------------------------end
start new k=3,train set=2630,validation set=292
**length fold set 267.0
**average fold coverage 0.06691729323308271
**time elapsed 7.458302021026611
------------------------------------end
start new k=4,train set=2630,validation set=292
**length fold set 292.0
**average fold coverage 0.07318295739348371
**time elapsed 9.225175142288208
------------------------------------end
start new k=5,train set=2630,validation set=292
**length fold set 256.0
**average fold coverage 0.06416040100250626
**time elapsed 11.090739965438843
------------------------------------end
start new k=6,train set=2630,validation set=292
**length fold set 256.0
**average fold coverage 0.064

start new k=8,train set=22500,validation set=2500
** number of elements in calculation 695
** average fold accuracy 0.6273381294964029
**time elapsed 645.0855438709259
------------------------------------end
start new k=9,train set=22500,validation set=2500
** number of elements in calculation 671
** average fold accuracy 0.6020864381520119
**time elapsed 717.6309440135956
------------------------------------end
list of total accuracy for each fold [0.60303030303030303, 0.59332321699544766, 0.60230547550432278, 0.59654178674351588, 0.63439306358381498, 0.64094955489614247, 0.61781609195402298, 0.62243285939968407, 0.62733812949640289, 0.6020864381520119]
10-fold total average 0.614021691975567
time elapsed 717.6408438682556
start new k=0,train set=22500,validation set=2500
**length fold set 741.0
**average fold coverage 0.18571428571428572
**time elapsed 68.27596712112427
------------------------------------end
start new k=1,train set=22500,validation set=2500
**length fold set 748.0
*

start new k=3,train set=12253,validation set=1362
** number of elements in calculation 342
** average fold accuracy 0.6169590643274854
**time elapsed 78.49179911613464
------------------------------------end
start new k=4,train set=12253,validation set=1362
** number of elements in calculation 340
** average fold accuracy 0.5411764705882353
**time elapsed 98.04972910881042
------------------------------------end
start new k=5,train set=12254,validation set=1361
** number of elements in calculation 390
** average fold accuracy 0.6153846153846154
**time elapsed 117.66157007217407
------------------------------------end
start new k=6,train set=12254,validation set=1361
** number of elements in calculation 375
** average fold accuracy 0.552
**time elapsed 137.01140904426575
------------------------------------end
start new k=7,train set=12254,validation set=1361
** number of elements in calculation 359
** average fold accuracy 0.5710306406685237
**time elapsed 156.55154705047607
----------

In [120]:
demo_accuracy

[(0, 4578, 0.51329617041439146),
 (1, 25000, 0.61074173494715467),
 (2, 9728, 0.55641090419339245),
 (3, 2922, 0.47887670121605119),
 (4, 23304, 0.60803886277844277),
 (5, 25000, 0.614021691975567),
 (6, 25000, 0.60783633248373437),
 (7, 13615, 0.58504224509857605)]

In [121]:
np.save('demo_clustering_cluster_accuracy3.npy',\
        [(maximum_rows,minimum_rows,seed,datetime.now().strftime("%Y-%m-%d_%H:%M:%S"))]+demo_accuracy)

In [122]:
demo_coverage

[(0, 4578, 0.099548872180451123),
 (1, 25000, 0.16854636591478697),
 (2, 9728, 0.11989974937343359),
 (3, 2922, 0.066240601503759405),
 (4, 23304, 0.17318295739348369),
 (5, 25000, 0.18639097744360902),
 (6, 25000, 0.17859649122807017),
 (7, 13615, 0.14147869674185462)]

In [123]:
np.save('demo_clustering_cluster_coverage3.npy',\
        [(maximum_rows,minimum_rows,seed,datetime.now().strftime("%Y-%m-%d_%H:%M:%S"))]+demo_coverage)